# CpGPTGrimAge3

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.CpGPTGrimAge3)

class CpGPTGrimAge3(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        """
        Scales an array based on the median and standard deviation.
        """
        median = torch.tensor(self.preprocess_dependencies[0], device=x.device, dtype=x.dtype)
        std = torch.tensor(self.preprocess_dependencies[1], device=x.device, dtype=x.dtype)
        x = (x - median) / std
        return x

    def postprocess(self, x):
        """
        Converts from a Cox parameter to age in units of years.
        """
        cox_mean = self.postprocess_dependencies[0]
        cox_std = self.postprocess_dependencies[1]
        age_mean = self.postprocess_dependencies[2]
        age_std = self.postprocess_dependencies[3]

        # Normalize
        x = (x - cox_mean) / cox_std

        # Scale
        x = (x * age_std) + age_mean

        return x



In [3]:
model = pya.models.CpGPTGrimAge3()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'cpgptgrimage3'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "de Lima Camillo, Lucas Paulo, et al. \"CpGPT: a foundation model for DNA methylation.\" bioRxiv (2024): 2024-10."
model.metadata["doi"] = "https://doi.org/10.1101/2024.10.24.619766"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [5]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.us-east-1.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/reliable/cpgpt_grimage3_weights_all_datasets_reliable.csv",
    "https://pyaging.s3.us-east-1.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/reliable/input_scaler_mean_all_datasets_reliable.npy",
    "https://pyaging.s3.us-east-1.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/reliable/input_scaler_scale_all_datasets_reliable.npy"
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Data found in ./cpgpt_grimage3_weights_all_datasets_reliable.csv
|-----------> Data found in ./input_scaler_mean_all_datasets_reliable.npy
|-----------> Data found in ./input_scaler_scale_all_datasets_reliable.npy


## Load features

#### From CSV

In [6]:
df = pd.read_csv('cpgpt_grimage3_weights_all_datasets_reliable.csv')
model.features = df['feature'].tolist()

In [7]:
df.head()

,feature,coefficient
0,age,0.845167
1,grimage2timp1,0.318954
2,grimage2packyrs,0.385882
3,grimage2logcrp,0.404675
4,grimage2adm,0.180551


## Load weights into base model

#### Linear model

In [8]:
weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([0.0])

#### Linear model

In [9]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [11]:
scale_mean = np.load('input_scaler_mean_all_datasets_reliable.npy')
scale_std = np.load('input_scaler_scale_all_datasets_reliable.npy')

model.reference_values = None

## Load preprocess and postprocess objects

In [12]:
model.preprocess_name = 'scale'
model.preprocess_dependencies = [scale_mean, scale_std]

In [13]:
model.postprocess_name = 'cox_to_years'
model.postprocess_dependencies = [
    0.54372919,
    1.52036698,
    64.94560376271838,
    11.920838151170104
]

## Check all clock parameters

In [14]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'de Lima Camillo, Lucas Paulo, et al. "CpGPT: a foundation model '
             'for DNA methylation." bioRxiv (2024): 2024-10.',
 'clock_name': 'cpgptgrimage3',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1101/2024.10.24.619766',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2025}
reference_values: None
preprocess_name: 'scale'
preprocess_dependencies: [array([ 6.50000000e+01,  3.49212152e+04,  1.21734902e+01,  2.73993813e-01,
        3.51222301e+02,  8.51761217e+03,  8.85501049e+02, -5.21484375e-01,
       -2.49755859e-01, -2.58056641e-01, -7.65991211e-02, -1.37939453e-01,
        2.53173828e-01,  1.33399963e-02, -4.01245117e-01,  1.90368652e-01,
       -3.27301025e-02,  1.29127502e-02, -2.78564453e-01,  1.92277772e+04,
        2.87399292e-02, -3.610

## Basic test

In [15]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[-425.9024],
        [-312.5518],
        [-563.8393],
        [-260.0897],
        [ -69.1534],
        [  30.5343],
        [-252.0608],
        [-445.2048],
        [ -64.2164],
        [ 103.0451]], dtype=torch.float64, grad_fn=<AddBackward0>)

## Save torch model

In [16]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [17]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: cpgpt_grimage3_weights_all_datasets_reliable.csv
Deleted file: input_scaler_mean_all_datasets_reliable.npy
Deleted file: input_scaler_scale_all_datasets_reliable.npy
